In [127]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
datas = pd.read_csv("train.csv")
datas.head()
datas.info()
y_col = "Survived"
x_cols = datas.columns.drop(["PassengerId","Survived","Name","Ticket","Fare","Cabin","Embarked"])
print(x_cols)
#!pip install autograd
#!pip install sklearn
from autograd import numpy
from autograd import grad

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch'], dtype='object')


In [128]:
for i in range(len(datas["Sex"])):
    if datas["Sex"][i] == "male":
        datas["Sex"][i] = 0
    else:
        datas["Sex"][i] = 1

mean_of_age=datas['Age'].mean()
isnalst = datas["Age"].isna()
for i in range(len(isnalst)):
    if isnalst[i]:
        datas["Age"][i] = mean_of_age

print(datas.head())

from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
X_scaled = min_max_scaler.fit_transform(datas[x_cols])
X_scaled = numpy.hstack((numpy.ones((X_scaled.shape[0],1)),X_scaled))

y = datas[y_col].values

print(f"x.s={X_scaled.shape} y.s={y.shape}")


c:\Program Files\Python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\Program Files\Python37\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name Sex   Age  SibSp  Parch  \
0                            Braund, Mr. Owen Harris   0  22.0      1      0   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...   1  38.0      1      0   
2                             Heikkinen, Miss. Laina   1  26.0      0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)   1  35.0      1      0   
4                           Allen, Mr. William Henry   0  35.0      0      0   

             Ticket     Fare Cabin Embarked  
0         A/5 21171   7.2500   NaN        S  
1          PC 17599  71.2833   C85        C  
2  STON/O2. 3101282   7.9250   NaN        S  
3            113803  53.1000  C123        S  
4            373450   8.0500   NaN        S  
x.s=(891, 6) y.s=(891,)


c:\Program Files\Python37\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [129]:
def logistic(z):
    return 1/(1+numpy.exp(-z))
def logistic_model(params,x):
    z = numpy.dot(X_scaled,params)
    y = logistic(z)
    return y
def log_loss(params,model,x,y,_lambda=1.0):
    y_pred = model(params,x)

    return -numpy.mean(y * numpy.log(y_pred) + (1-y) * numpy.log(1 - y_pred)) \
+ _lambda * numpy.sum(params[1:]**2)

In [130]:
gradient = grad(log_loss)

In [131]:
type(gradient)

function

In [132]:
max_iter = 3000
alpha = 0.01
params = numpy.zeros(X_scaled.shape[1])
descent = numpy.ones(X_scaled.shape[1])
i = 0
print(params)
while (numpy.linalg.norm(descent) > 0.001 and i<max_iter):
    descent = gradient(params,logistic_model,X_scaled,y)
    params = params - descent * alpha
    i += 1
    if i%100 == 0:
        print(f"--{params}--")

[0. 0. 0. 0. 0. 0.]
--[-0.09821959 -0.05543168  0.03956746 -0.01736389 -0.00367905 -0.00031607]--
--[-0.17321889 -0.05650497  0.04766804 -0.01628809 -0.00357112  0.00020316]--
--[-0.23269811 -0.05248519  0.05092101 -0.01377295 -0.00314106  0.00067378]--
--[-0.28016377 -0.04873687  0.05308051 -0.0115883  -0.00275978  0.00105107]--
--[-0.31811304 -0.04569343  0.05474253 -0.00983036 -0.00245199  0.00135113]--
--[-0.34848969 -0.0432632   0.05605948 -0.00842843 -0.00220637  0.00159007]--
--[-0.37282836 -0.04132458  0.05710938 -0.00731034 -0.00201043  0.00178063]--
--[-0.39234523 -0.03977639  0.05794818 -0.00641749 -0.00185394  0.00193284]--
--[-0.40800635 -0.03853841  0.05861921 -0.00570357 -0.00172879  0.00205458]--
--[-0.42058069 -0.03754735  0.05915662 -0.00513208 -0.00162859  0.00215204]--
--[-0.43068144 -0.03675317  0.05958739 -0.00467413 -0.0015483   0.00223016]--
--[-0.43879837 -0.03611625  0.05993296 -0.00430688 -0.00148391  0.00229281]--
--[-0.44532317 -0.03560509  0.06021035 -0.00

In [133]:
def decision_boundary(y):
    return 1 if y>=0.5 else 0

In [134]:
decision_boundary = np.vectorize(decision_boundary)

In [137]:
def classify(predictions):
    return decision_boundary(predictions).flatten()

tdatas=pd.read_csv("test.csv")
xt_cols = tdatas.columns.drop(["PassengerId","Name","Ticket","Fare","Cabin","Embarked"])

for i in range(len(tdatas["Sex"])):
    if tdatas["Sex"][i] == "male":
        tdatas["Sex"][i] = 0
    else:
        tdatas["Sex"][i] = 1

mean_of_age=tdatas['Age'].mean()
isnalst = tdatas["Age"].isna()
for i in range(len(isnalst)):
    if isnalst[i]:
        tdatas["Age"][i] = mean_of_age

Xt_scaled = min_max_scaler.fit_transform(tdatas[xt_cols])
Xt_scaled = numpy.hstack((numpy.ones((Xt_scaled.shape[0],1)),Xt_scaled))
dec = classify(logistic_model(params,Xt_scaled))
print(dec)

FileNotFoundError: [Errno 2] No such file or directory: 'titanic/test.csv'